# US Golf Course Information Kiosk

Detailed information on golf courses with green fees range and tees, this complete golf courses database has 15,606 records across 7,891 cities over 53 states in the United States. Each record is comprised of address, street, phone number, zip code, hole, architect, year built, public/private, guest policy, credit card, golf season, range, rental club, pro in house, metal spikes okay, weekday, weekend, tee time welcomed, rental cart available, championship par/yards/slope/USGA, middle par/yards/slope/USGA and forward par/yards/slope/USGA

https://www.usabledatabases.com/database/golf-courses-in-us/

## Step 1:  Clean and transform the input dataset

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import json
from sqlalchemy import create_engine
from config import db_connection

# Import API key
from config import gkey

# CSV files
course = "data/course.csv"
city = "data/city.csv"
state = "data/state.csv"

In [2]:
# Create the data frame - golf courses
course_df = pd.read_csv(course, 
                        delimiter=',', 
                        skipinitialspace=True, 
                        dtype={"zip_code":object, "hole":object })

# Add columns for longitude and latitude
course_df["lng"] = ""
course_df["lat"] = ""

golf_courses = course_df.rename(columns={"id": "course_id",
                                         "title": "course"})
golf_courses

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat
0,1,1,Albertville Golf & Country Club,Country Club Rd,"Albertville, Alabama 35950",35950,(256) 878-4403,18,Leon Howard,1966,...,72,6068,117,68.00,72,5196,NaN,NaN,,
1,2,1,A. J. Jolly Golf Course,5350 S US Hwy 27,"Alexandria, Kentucky 41001",41001,(606) 635-2106,18,NaN,1962,...,71,5942,115,67.60,75,5418,118,70.30,,
2,3,2,Willow Point Country Club,3054 Willow Point Rd,"Alexander City, Alabama 35010",35010,(256) 234-2572,18,Thomas H. Nicol,1961,...,72,6631,135,72.20,73,5373,120,70.70,,
3,4,3,Alpine Bay Resort,9855 Renfroe Rd,"Alpine, Alabama 35014",35014,(256) 268-2920,18,"Robert Trent Jones, Sr.",1972,...,72,6518,126,69.90,72,5518,120,69.80,,
4,5,4,Maple Hills Golf,Hwy 75,"Altoona, Alabama 35952",35952,(205) 466-7600,9,Bill Ellison,1975,...,31,1800,NaN,NaN,31,1800,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15601,15750,8219,Legion Town & Country Club,141 Airport Rd,"Thermopolis, Wyoming 82443",82443,(307) 864-5294,9,Don Brunk,1962,...,35,2966,111,33.90,37,2836,111,35.00,,
15602,15751,8220,Cottonwood Country Club,15th St,"Torrington, Wyoming 82240",82240,(307) 532-3868,18,NaN,NaN,...,72,6298,112,67.70,73,5344,112,69.20,,
15603,15752,8221,Cedar Pines Golf Course,2579 N Hwy 116,"Upton, Wyoming 82730",82730,(307) 468-2847,9,NaN,1984,...,36,3198,NaN,NaN,38,2521,NaN,NaN,,
15604,15753,8222,Wheatland Golf Club,1253 E Cole,"Wheatland, Wyoming 82201",82201,(307) 322-3675,9,NaN,1960,...,36,3064,119,33.95,36,2557,116,34.25,,


In [3]:
# Checking for duplicate rows in golf courses
duplicateRow = golf_courses[golf_courses.duplicated()]
duplicateRow

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat


In [5]:
# Create a subset data frame - MN golf courses
subset_df = golf_courses[((golf_courses["zip_code"] >= "55001") & (golf_courses["zip_code"] <= "56763"))]

subset_df

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,middle_par,middle_yards,middle_slope,middle_usga,forward_par,forward_yards,forward_slope,forward_usga,lng,lat
6960,7039,3335,Otsego Club And Resort,696 M-32 East Main Street,P.O. Box 556,556,NaN,18,William H. Diddle,1955,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
7584,7671,3612,Heart of the Valley Golf Club,SE Corner of Ada,"Ada, Minnesota 56510",56510,(218) 784-4746,9,NaN,1960,...,36,2994,NaN,NaN,37,2560,NaN,NaN,,
7585,7672,3613,Cedar River Country Club,Hwy 56 W,"Adams, Minnesota 55909",55909,(507) 582-3595,18,NaN,1965,...,72,6076,117,68.40,74,5553,119,71.30,,
7586,7673,3614,Adrian Golf Course,Hwy 91 S,"Adrian, Minnesota 56110",56110,(507) 483-2722,9,Drew Demarest,1994,...,36,3140,120,35.00,36,2601,115,34.80,,
7587,7674,3615,Albany Golf Club,500 Church St,"Albany, Minnesota 56307",56307,(320) 845-2505,18,Willie Kidd,1960,...,72,6023,119,68.20,74,5268,117,69.70,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8063,8155,3900,Prairie View Golf Links,Hwy 266 No,"Worthington, Minnesota 56187",56187,(507) 372-8670,18,Joel Goldstrand,1983,...,71,5989,111,68.30,71,5105,111,69.00,,
8064,8156,3900,Worthington Country Club,1414 Liberty Dr,"Worthington, Minnesota 56187",56187,(507) 376-4281,18,Leo Johnson,1919,...,71,5982,123,68.60,71,5243,122,70.80,,
8065,8157,3901,"Greenwood Golf Links, Inc.",4520 E Viking Blvd,"Wyoming, Minnesota 55092",55092,(651) 462-4653,18,C. M. Johnson,1985,...,72,5438,103,66.50,72,4791,112,67.30,,
8066,8158,3902,Fairway Shores Golf Course,28340 131St St,"Zimmerman, Minnesota 55398",55398,(612) 856-3334,9,Dan Springman,1998,...,31,1673,NaN,NaN,31,1673,NaN,NaN,,


In [ ]:
# Create a subset data frame - variety of golf courses
bool_series = golf_courses["zip_code"].str.startswith("010", na = False) 
df1 = golf_courses[bool_series]
bool_series = golf_courses["zip_code"].str.startswith("6000", na = False) 
df2 = golf_courses[bool_series]
bool_series = golf_courses["zip_code"].str.startswith("5600", na = False) 
df3 = golf_courses[bool_series]
bool_series = golf_courses["zip_code"].str.startswith("920", na = False) 
df4 = golf_courses[bool_series]

subset_df = df1.append(df2, ignore_index = True)
subset_df = subset_df.append(df3, ignore_index = True)
subset_df = subset_df.append(df4, ignore_index = True)
subset_df

In [6]:
# Create the data frame - golf cities
city_df = pd.read_csv(city, delimiter=',', skipinitialspace=True)

golf_cities = city_df.rename(columns={"id": "city_id", 
                                      "title": "city_title", 
                                      "count": "city_count", 
                                      "slug": "city_slug"})
golf_cities

,city_id,state_id,city_title,city_count,city_slug
0,1,1,Albertville,2,albertville
1,2,1,Alexander City,1,alexandercity
2,3,1,Alpine,1,alpine
3,4,1,Altoona,1,altoona
4,5,1,Andalusia,2,andalusia
...,...,...,...,...,...
7886,8219,53,Thermopolis,1,thermopolis
7887,8220,53,Torrington,1,torrington
7888,8221,53,Upton,1,upton
7889,8222,53,Wheatland,1,wheatland


In [7]:
# Checking for duplicate rows in golf cities
duplicateRow = golf_cities[golf_cities.duplicated()]
duplicateRow

,city_id,state_id,city_title,city_count,city_slug


In [8]:
# Create the data frame - golf states
state_df = pd.read_csv(state, delimiter=',', skipinitialspace=True)

golf_states = state_df.rename(columns={"id": "state_id", 
                                       "title": "state", 
                                       "count": "state_count", 
                                       "slug": "state_slug"})
golf_states

,state_id,state,state_count,state_slug
0,1,Alabama,272,alabama
1,2,Alaska,16,alaska
2,3,Arizona,308,arizona
3,4,Arkansas,179,arkansas
4,5,California,976,california
5,6,Colorado,223,colorado
6,7,Connecticut,177,connecticut
7,8,Delaware,39,delaware
8,9,District of Columbia,6,districtofcolumbia
9,10,Florida,1094,florida


In [9]:
# Checking for duplicate rows in golf states
duplicateRow = golf_states[golf_states.duplicated()]
duplicateRow

,state_id,state,state_count,state_slug


### Merge the data frames to create one complete dataset

In [10]:
# Merge the city and state dataframes using a left join
city_state_df = pd.merge(golf_cities, golf_states, on="state_id", how="left")
city_state_df

,city_id,state_id,city_title,city_count,city_slug,state,state_count,state_slug
0,1,1,Albertville,2,albertville,Alabama,272,alabama
1,2,1,Alexander City,1,alexandercity,Alabama,272,alabama
2,3,1,Alpine,1,alpine,Alabama,272,alabama
3,4,1,Altoona,1,altoona,Alabama,272,alabama
4,5,1,Andalusia,2,andalusia,Alabama,272,alabama
...,...,...,...,...,...,...,...,...
7886,8219,53,Thermopolis,1,thermopolis,Wyoming,48,wyoming
7887,8220,53,Torrington,1,torrington,Wyoming,48,wyoming
7888,8221,53,Upton,1,upton,Wyoming,48,wyoming
7889,8222,53,Wheatland,1,wheatland,Wyoming,48,wyoming


In [11]:
# Merge the city_state dataframe with the golf course dataframe using a left join
golf_citystate = pd.merge(subset_df, city_state_df, on="city_id" ,how="left")
golf_citystate

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,forward_usga,lng,lat,state_id,city_title,city_count,city_slug,state,state_count,state_slug
0,7039,3335,Otsego Club And Resort,696 M-32 East Main Street,P.O. Box 556,556,NaN,18,William H. Diddle,1955,...,NaN,,,23,Gaylord,14,gaylord,Michigan,933,michigan
1,7671,3612,Heart of the Valley Golf Club,SE Corner of Ada,"Ada, Minnesota 56510",56510,(218) 784-4746,9,NaN,1960,...,NaN,,,24,Ada,1,ada,Minnesota,484,minnesota
2,7672,3613,Cedar River Country Club,Hwy 56 W,"Adams, Minnesota 55909",55909,(507) 582-3595,18,NaN,1965,...,71.30,,,24,Adams,1,adams,Minnesota,484,minnesota
3,7673,3614,Adrian Golf Course,Hwy 91 S,"Adrian, Minnesota 56110",56110,(507) 483-2722,9,Drew Demarest,1994,...,34.80,,,24,Adrian,1,adrian,Minnesota,484,minnesota
4,7674,3615,Albany Golf Club,500 Church St,"Albany, Minnesota 56307",56307,(320) 845-2505,18,Willie Kidd,1960,...,69.70,,,24,Albany,1,albany,Minnesota,484,minnesota
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,8155,3900,Prairie View Golf Links,Hwy 266 No,"Worthington, Minnesota 56187",56187,(507) 372-8670,18,Joel Goldstrand,1983,...,69.00,,,24,Worthington,2,worthington,Minnesota,484,minnesota
481,8156,3900,Worthington Country Club,1414 Liberty Dr,"Worthington, Minnesota 56187",56187,(507) 376-4281,18,Leo Johnson,1919,...,70.80,,,24,Worthington,2,worthington,Minnesota,484,minnesota
482,8157,3901,"Greenwood Golf Links, Inc.",4520 E Viking Blvd,"Wyoming, Minnesota 55092",55092,(651) 462-4653,18,C. M. Johnson,1985,...,67.30,,,24,Wyoming,1,wyoming,Minnesota,484,minnesota
483,8158,3902,Fairway Shores Golf Course,28340 131St St,"Zimmerman, Minnesota 55398",55398,(612) 856-3334,9,Dan Springman,1998,...,NaN,,,24,Zimmerman,1,zimmerman,Minnesota,484,minnesota


In [12]:
regex = r'(?P<c_city>[^,]+)\s*,\s*'
golf_address = golf_citystate.join(golf_citystate["address"].str.extract(regex))
golf_address

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,lng,lat,state_id,city_title,city_count,city_slug,state,state_count,state_slug,c_city
0,7039,3335,Otsego Club And Resort,696 M-32 East Main Street,P.O. Box 556,556,NaN,18,William H. Diddle,1955,...,,,23,Gaylord,14,gaylord,Michigan,933,michigan,NaN
1,7671,3612,Heart of the Valley Golf Club,SE Corner of Ada,"Ada, Minnesota 56510",56510,(218) 784-4746,9,NaN,1960,...,,,24,Ada,1,ada,Minnesota,484,minnesota,Ada
2,7672,3613,Cedar River Country Club,Hwy 56 W,"Adams, Minnesota 55909",55909,(507) 582-3595,18,NaN,1965,...,,,24,Adams,1,adams,Minnesota,484,minnesota,Adams
3,7673,3614,Adrian Golf Course,Hwy 91 S,"Adrian, Minnesota 56110",56110,(507) 483-2722,9,Drew Demarest,1994,...,,,24,Adrian,1,adrian,Minnesota,484,minnesota,Adrian
4,7674,3615,Albany Golf Club,500 Church St,"Albany, Minnesota 56307",56307,(320) 845-2505,18,Willie Kidd,1960,...,,,24,Albany,1,albany,Minnesota,484,minnesota,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,8155,3900,Prairie View Golf Links,Hwy 266 No,"Worthington, Minnesota 56187",56187,(507) 372-8670,18,Joel Goldstrand,1983,...,,,24,Worthington,2,worthington,Minnesota,484,minnesota,Worthington
481,8156,3900,Worthington Country Club,1414 Liberty Dr,"Worthington, Minnesota 56187",56187,(507) 376-4281,18,Leo Johnson,1919,...,,,24,Worthington,2,worthington,Minnesota,484,minnesota,Worthington
482,8157,3901,"Greenwood Golf Links, Inc.",4520 E Viking Blvd,"Wyoming, Minnesota 55092",55092,(651) 462-4653,18,C. M. Johnson,1985,...,,,24,Wyoming,1,wyoming,Minnesota,484,minnesota,Wyoming
483,8158,3902,Fairway Shores Golf Course,28340 131St St,"Zimmerman, Minnesota 55398",55398,(612) 856-3334,9,Dan Springman,1998,...,,,24,Zimmerman,1,zimmerman,Minnesota,484,minnesota,Zimmerman


In [13]:
# Finding the best city to use
golf_address["city"] = golf_address.c_city.combine_first(golf_address.city_title)

golf_df = golf_address
golf_df

,course_id,city_id,course,street,address,zip_code,phone,hole,architect,year_built,...,lat,state_id,city_title,city_count,city_slug,state,state_count,state_slug,c_city,city
0,7039,3335,Otsego Club And Resort,696 M-32 East Main Street,P.O. Box 556,556,NaN,18,William H. Diddle,1955,...,,23,Gaylord,14,gaylord,Michigan,933,michigan,NaN,Gaylord
1,7671,3612,Heart of the Valley Golf Club,SE Corner of Ada,"Ada, Minnesota 56510",56510,(218) 784-4746,9,NaN,1960,...,,24,Ada,1,ada,Minnesota,484,minnesota,Ada,Ada
2,7672,3613,Cedar River Country Club,Hwy 56 W,"Adams, Minnesota 55909",55909,(507) 582-3595,18,NaN,1965,...,,24,Adams,1,adams,Minnesota,484,minnesota,Adams,Adams
3,7673,3614,Adrian Golf Course,Hwy 91 S,"Adrian, Minnesota 56110",56110,(507) 483-2722,9,Drew Demarest,1994,...,,24,Adrian,1,adrian,Minnesota,484,minnesota,Adrian,Adrian
4,7674,3615,Albany Golf Club,500 Church St,"Albany, Minnesota 56307",56307,(320) 845-2505,18,Willie Kidd,1960,...,,24,Albany,1,albany,Minnesota,484,minnesota,Albany,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,8155,3900,Prairie View Golf Links,Hwy 266 No,"Worthington, Minnesota 56187",56187,(507) 372-8670,18,Joel Goldstrand,1983,...,,24,Worthington,2,worthington,Minnesota,484,minnesota,Worthington,Worthington
481,8156,3900,Worthington Country Club,1414 Liberty Dr,"Worthington, Minnesota 56187",56187,(507) 376-4281,18,Leo Johnson,1919,...,,24,Worthington,2,worthington,Minnesota,484,minnesota,Worthington,Worthington
482,8157,3901,"Greenwood Golf Links, Inc.",4520 E Viking Blvd,"Wyoming, Minnesota 55092",55092,(651) 462-4653,18,C. M. Johnson,1985,...,,24,Wyoming,1,wyoming,Minnesota,484,minnesota,Wyoming,Wyoming
483,8158,3902,Fairway Shores Golf Course,28340 131St St,"Zimmerman, Minnesota 55398",55398,(612) 856-3334,9,Dan Springman,1998,...,,24,Zimmerman,1,zimmerman,Minnesota,484,minnesota,Zimmerman,Zimmerman


In [14]:
# Select the golf course columns needed for course longitude, latitude
golf_course_df = golf_df[["course_id",
                          "course",
                          "city",
                          "state",
                          "street",
                          "zip_code",
                          "lng", 
                          "lat",
                          "hole",
                          "public_private"]]
                     
# Display the new dataframe
golf_course_df

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private
0,7039,Otsego Club And Resort,Gaylord,Michigan,696 M-32 East Main Street,556,,,18,Public
1,7671,Heart of the Valley Golf Club,Ada,Minnesota,SE Corner of Ada,56510,,,9,Semi-Private
2,7672,Cedar River Country Club,Adams,Minnesota,Hwy 56 W,55909,,,18,Semi-Private
3,7673,Adrian Golf Course,Adrian,Minnesota,Hwy 91 S,56110,,,9,Semi-Private
4,7674,Albany Golf Club,Albany,Minnesota,500 Church St,56307,,,18,Public
...,...,...,...,...,...,...,...,...,...,...
480,8155,Prairie View Golf Links,Worthington,Minnesota,Hwy 266 No,56187,,,18,Public
481,8156,Worthington Country Club,Worthington,Minnesota,1414 Liberty Dr,56187,,,18,Semi-Private
482,8157,"Greenwood Golf Links, Inc.",Wyoming,Minnesota,4520 E Viking Blvd,55092,,,18,Public
483,8158,Fairway Shores Golf Course,Zimmerman,Minnesota,28340 131St St,55398,,,9,Public


In [15]:
# Select the golf course columns needed for golf kiosk - general information
course_info = golf_df[["course_id",
                       "phone",
                       "architect",
                       "year_built",
                       "guest_policy",
                       "credit_card"]]
                     
# Display the new dataframe
course_info

,course_id,phone,architect,year_built,guest_policy,credit_card
0,7039,NaN,William H. Diddle,1955,Open/ Group Rates available,Amex MasterCard Visa card(s) Welcomed
1,7671,(218) 784-4746,NaN,1960,Open,None
2,7672,(507) 582-3595,NaN,1965,Open,"VISA, MasterCard Welcomed"
3,7673,(507) 483-2722,Drew Demarest,1994,Open,"VISA, MasterCard Welcomed"
4,7674,(320) 845-2505,Willie Kidd,1960,Open,"VISA, MasterCard Welcomed"
...,...,...,...,...,...,...
480,8155,(507) 372-8670,Joel Goldstrand,1983,Open,"VISA, MasterCard Welcomed"
481,8156,(507) 376-4281,Leo Johnson,1919,Open,None
482,8157,(651) 462-4653,C. M. Johnson,1985,Open,None
483,8158,(612) 856-3334,Dan Springman,1998,Open,None


In [16]:
# Select the golf course columns needed for golf kiosk - course details
course_details = golf_df[["course_id",
                          "golf_season",
                          "range",
                          "rental_club",
                          "pro_in_House",
                          "metal_spikes_okay",
                          "weekday",
                          "weekend",
                          "tee_time_welcomed",
                          "rental_cart_available"]]
                     
# Display the new dataframe
course_details

,course_id,golf_season,range,rental_club,pro_in_House,metal_spikes_okay,weekday,weekend,tee_time_welcomed,rental_cart_available
0,7039,Open: 4/15 Closed: 10/15,Yes,Yes,Yes,No,$,$,&nbsp;Yes,Yes
1,7671,Open: 4/15 Closed: 10/15,No,Yes,Yes,No,$,$,No,Yes
2,7672,Open: 4/01 Closed: 11/01,Yes,Yes,Yes,No,$,$,Yes,Yes
3,7673,Open: 5/01 Closed: 10/01,Yes,No,No,No,$,$,Yes,Yes
4,7674,Open: 4/01 Closed: 11/01,Yes,Yes,Yes,No,$,$,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...
480,8155,Open: 4/01 Closed: 11/01,Yes,Yes,Yes,No,$,$,Yes,Yes
481,8156,Open: 4/01 Closed: 11/01,Yes,No,Yes,No,$,$,Yes,Yes
482,8157,Open: 4/15 Closed: 11/01,No,Yes,No,Yes,$,$,Yes,Yes
483,8158,Open: 4/10 Closed: 10/01,No,Yes,No,No,$,$,Yes,Yes


In [17]:
# Select the golf course columns needed for golf kiosk - championship tees
championship_tees = golf_df[["course_id", 
                            "championship_par", 
                            "championship_yards", 
                            "championship_slope", 
                            "championship_usga"]]
                     
# Display the new dataframe
championship_tees

,course_id,championship_par,championship_yards,championship_slope,championship_usga
0,7039,NaN,NaN,NaN,NaN
1,7671,NaN,NaN,NaN,NaN
2,7672,72,6298,119,69.30
3,7673,36,3345,123,35.80
4,7674,72,6415,122,70.00
...,...,...,...,...,...
480,8155,71,6368,114,70.10
481,8156,71,6276,125,70.00
482,8157,72,5518,105,67.20
483,8158,NaN,NaN,NaN,NaN


In [18]:
# Select the golf course columns needed for golf kiosk - middle tees
middle_tees = golf_df[["course_id",
                       "middle_par",
                       "middle_yards",
                       "middle_slope",
                       "middle_usga"]]
                     
# Display the new dataframe
middle_tees

,course_id,middle_par,middle_yards,middle_slope,middle_usga
0,7039,NaN,NaN,NaN,NaN
1,7671,36,2994,NaN,NaN
2,7672,72,6076,117,68.40
3,7673,36,3140,120,35.00
4,7674,72,6023,119,68.20
...,...,...,...,...,...
480,8155,71,5989,111,68.30
481,8156,71,5982,123,68.60
482,8157,72,5438,103,66.50
483,8158,31,1673,NaN,NaN


In [19]:
# Select the golf course columns needed for golf kiosk - forward tees
forward_tees = golf_df[["course_id",
                        "forward_par",
                        "forward_yards",
                        "forward_slope",
                        "forward_usga"]]
                     
# Display the new dataframe
forward_tees

,course_id,forward_par,forward_yards,forward_slope,forward_usga
0,7039,NaN,NaN,NaN,NaN
1,7671,37,2560,NaN,NaN
2,7672,74,5553,119,71.30
3,7673,36,2601,115,34.80
4,7674,74,5268,117,69.70
...,...,...,...,...,...
480,8155,71,5105,111,69.00
481,8156,71,5243,122,70.80
482,8157,72,4791,112,67.30
483,8158,31,1673,NaN,NaN


In [20]:
# Getting the longitude and latitude for each golf course
count = 0

# Use the street, city, state to get lng and lat
for index, row in golf_course_df.iterrows():
    # get street, city, state from golf_course_df
    street = row["street"]
    city = row["city"]
    state = row["state"]
    postal_code = row["zip_code"]

    # Getting lat/lng
    query_address_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={street},+{city},+{state}&key={gkey}"
    # print(f"url = {query_address_url}")
        
    query_zip_url = f"https://maps.googleapis.com/maps/api/geocode/json?components=postal_code:{postal_code}&key={gkey}"
    # print(f"url = {query_zip_url}")

    response = requests.get(query_address_url).json()
    # print(f"response = {response}")
    
    response_zip = requests.get(query_zip_url).json()
    # print(f"response_zip = {response_zip}")
    
    count = count + 1
    # print(f"#{count} - index = {index}")
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        golf_course_df.loc[index, "lat"] = response["results"][0]["geometry"]["location"]["lat"]
        golf_course_df.loc[index, "lng"] = response["results"][0]["geometry"]["location"]["lng"]
        
    except (KeyError, IndexError):
        golf_course_df.loc[index, "lat"] = response_zip["results"][0]["geometry"]["location"]["lat"]
        golf_course_df.loc[index, "lng"] = response_zip["results"][0]["geometry"]["location"]["lng"]
        print("Using zip code for lng, lat.")
        # print("Missing field/result... skipping.")

C:\Users\Public\anaconda3\envs\NewPythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Using zip code for lng, lat.


In [22]:
# Preview course subset
golf_course_df

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private
0,7039,Otsego Club And Resort,Gaylord,Michigan,696 M-32 East Main Street,556,-84.6553,45.0277,18,Public
1,7671,Heart of the Valley Golf Club,Ada,Minnesota,SE Corner of Ada,56510,-96.5153,47.2997,9,Semi-Private
2,7672,Cedar River Country Club,Adams,Minnesota,Hwy 56 W,55909,-92.7193,43.5654,18,Semi-Private
3,7673,Adrian Golf Course,Adrian,Minnesota,Hwy 91 S,56110,-95.9331,43.635,9,Semi-Private
4,7674,Albany Golf Club,Albany,Minnesota,500 Church St,56307,-94.5703,45.6291,18,Public
...,...,...,...,...,...,...,...,...,...,...
480,8155,Prairie View Golf Links,Worthington,Minnesota,Hwy 266 No,56187,-95.7137,43.7012,18,Public
481,8156,Worthington Country Club,Worthington,Minnesota,1414 Liberty Dr,56187,-95.6228,43.6255,18,Semi-Private
482,8157,"Greenwood Golf Links, Inc.",Wyoming,Minnesota,4520 E Viking Blvd,55092,-93.0147,45.3406,18,Public
483,8158,Fairway Shores Golf Course,Zimmerman,Minnesota,28340 131St St,55398,-93.6016,45.4848,9,Public


In [23]:
# Create a dataframe to export
golf_courses = golf_course_df[["course_id",
                               "course",
                               "city",
                               "state",
                               "street",
                               "zip_code",
                               "lng",
                               "lat",
                               "hole",
                               "public_private"]]
                     
# Display the new dataframe
golf_courses

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private
0,7039,Otsego Club And Resort,Gaylord,Michigan,696 M-32 East Main Street,556,-84.6553,45.0277,18,Public
1,7671,Heart of the Valley Golf Club,Ada,Minnesota,SE Corner of Ada,56510,-96.5153,47.2997,9,Semi-Private
2,7672,Cedar River Country Club,Adams,Minnesota,Hwy 56 W,55909,-92.7193,43.5654,18,Semi-Private
3,7673,Adrian Golf Course,Adrian,Minnesota,Hwy 91 S,56110,-95.9331,43.635,9,Semi-Private
4,7674,Albany Golf Club,Albany,Minnesota,500 Church St,56307,-94.5703,45.6291,18,Public
...,...,...,...,...,...,...,...,...,...,...
480,8155,Prairie View Golf Links,Worthington,Minnesota,Hwy 266 No,56187,-95.7137,43.7012,18,Public
481,8156,Worthington Country Club,Worthington,Minnesota,1414 Liberty Dr,56187,-95.6228,43.6255,18,Semi-Private
482,8157,"Greenwood Golf Links, Inc.",Wyoming,Minnesota,4520 E Viking Blvd,55092,-93.0147,45.3406,18,Public
483,8158,Fairway Shores Golf Course,Zimmerman,Minnesota,28340 131St St,55398,-93.6016,45.4848,9,Public


In [24]:
# Export the golf course data to csv
golf_courses.zip_code = golf_courses.zip_code.apply('="{}"'.format)
golf_courses.to_csv("results/course_subset.csv", index=False, header=True)

In [25]:
golf_course_df

,course_id,course,city,state,street,zip_code,lng,lat,hole,public_private
0,7039,Otsego Club And Resort,Gaylord,Michigan,696 M-32 East Main Street,556,-84.6553,45.0277,18,Public
1,7671,Heart of the Valley Golf Club,Ada,Minnesota,SE Corner of Ada,56510,-96.5153,47.2997,9,Semi-Private
2,7672,Cedar River Country Club,Adams,Minnesota,Hwy 56 W,55909,-92.7193,43.5654,18,Semi-Private
3,7673,Adrian Golf Course,Adrian,Minnesota,Hwy 91 S,56110,-95.9331,43.635,9,Semi-Private
4,7674,Albany Golf Club,Albany,Minnesota,500 Church St,56307,-94.5703,45.6291,18,Public
...,...,...,...,...,...,...,...,...,...,...
480,8155,Prairie View Golf Links,Worthington,Minnesota,Hwy 266 No,56187,-95.7137,43.7012,18,Public
481,8156,Worthington Country Club,Worthington,Minnesota,1414 Liberty Dr,56187,-95.6228,43.6255,18,Semi-Private
482,8157,"Greenwood Golf Links, Inc.",Wyoming,Minnesota,4520 E Viking Blvd,55092,-93.0147,45.3406,18,Public
483,8158,Fairway Shores Golf Course,Zimmerman,Minnesota,28340 131St St,55398,-93.6016,45.4848,9,Public
